In [12]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.utils import np_utils

In [13]:
train_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/train.csv')
test_df = pd.read_csv('D:/Program/dataset/Spooky_Author_Identification/test.csv')


y_train = train_df["author"].values

In [14]:
with open('keras_input_train.pkl', 'rb') as f:
    sequences = pickle.load(f)
with open('keras_input_test.pkl', 'rb') as f:
    test_sequences = pickle.load(f)
    
x_train = sequences
x_test = test_sequences

print(x_train.shape)
print(x_test.shape)

(19579, 24992)
(8392, 24992)


In [15]:
gnb =  MultinomialNB()
gnb.fit(x_train.todense(), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
y_pred = gnb.predict(x_train.todense())
s = accuracy_score(y_train,y_pred)
print("The accuracy score {}".format(s))

The accuracy score 0.9110781960263548


In [17]:
print(gnb.classes_)
predict_probability = gnb.predict_proba(x_test)
print(predict_probability.shape)

['EAP' 'HPL' 'MWS']
(8392, 3)


In [18]:
# submission

submission = pd.DataFrame(predict_probability, columns=gnb.classes_)
submission["id"] = test_df["id"]
cols = submission.columns.tolist()
cols = cols[-1:]+cols[:-1]
submission = submission[cols]
submission.head(10)

,id,EAP,HPL,MWS
0,id02310,0.265857,0.096079,0.638064
1,id24541,0.766765,0.139109,0.094126
2,id00134,0.496714,0.392546,0.110740
3,id27757,0.560449,0.384442,0.055109
4,id04081,0.745861,0.149537,0.104602
5,id27337,0.593558,0.287258,0.119183
6,id24265,0.523722,0.323017,0.153260
7,id25917,0.149492,0.228221,0.622287
8,id04951,0.851826,0.109595,0.038579
9,id14549,0.624012,0.170226,0.205761


In [19]:
submission.to_csv('submission_NB.csv',index=False)